In [53]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
import datetime
from os import listdir
from os.path import isfile, join
percorso = 'D:/Università/Tirocinio/Analisi Dati/Studio Dati/'
onlyfiles = [f for f in listdir(f'{percorso}/risultati_terremoto/') if isfile(join(f'{percorso}/risultati_terremoto/', f))]
inizio = 20150413
fine = 20150418
comune = 'Abbateggio'

coms = pd.Series(pd.read_csv('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/sezione2comuneDEF.csv').comune.unique(), name= 'Comune')
crateri = pd.read_csv('D:/Università/Tirocinio/Analisi Dati/Studio Dati/crateriCentroIT.csv', sep=';').merge(coms, how='outer').drop(columns='Provincia')
crateri['Cratere Centro Italia 2016'].fillna('0', inplace=True)
crateri.rename(columns)

In [86]:
for i in onlyfiles:
    if (int(i.split('_')[1])>= inizio) & (int(i.split('_')[1]) <= fine):
        esempio = pd.read_csv(f'{percorso}/risultati_terremoto/CentroItalia_20150401_20150430_24.csv',header= None).rename(columns= {0: 'Comune', 1: 'val', 2: 'data'}, inplace = True)
        esempio = esempio.loc[esempio['Comune']== comune]
        esempio = crateri.merge(esempio, how= 'outer')
        esempio['Cratere Centro Italia 2016'].fillna(0, inplace= True)
        esempio.drop(columns= 'Provincia', inplace = True)
        esempio.rename(columns={'Cratere Centro Italia 2016': 'terremotata', 'Comune' : 'comune'}, inplace= True)
        esempio = esempio.sort_values(['comune', 'data'])
        esempio['data']= pd.to_datetime(esempio['data'], format='%Y%m%d_%H%M')

processo
processo
processo
processo
processo
processo


IndexError: list index out of range

In [23]:
esempio = pd.read_csv(f'{percorso}/risultati_terremoto/CentroItalia_20150401_20150430_24.csv', header=None)
esempio.rename(columns={0: 'Comune', 1: 'val', 2: 'data'}, inplace=True)
esempio = esempio.loc[esempio['Comune'] == comune]
# esempio = crateri.merge(esempio, how='outer')

,Comune,Cratere Centro Italia 2016
0,Cerreto d'Esi,1.0
1,Fabriano,1.0
2,Acquasanta Terme,1.0
3,Appignano del Tronto,1.0
4,Arquata del Tronto,1.0
...,...,...
658,Sante Marie,0
659,Roccacasale,0
660,Montorio Romano,0
661,Manziana,0
